In [1]:
import numpy as np
import pandas as pd

In [2]:
def get_pos_list(start, n_step, step_size):
    pos_list = []
    for i in range(n_step):
        pos_list.append(round((start + i * step_size), 4))
    return pos_list

In [3]:
motor1 = "CG1D:Mot:G0:X"
# motor2 = "CG1D:Mot:G0:Roll"
motor2 = None
# motor3 = "CG1D:Mot:G1:Roll"
motor3 = None

fname = "G0_stepping"
delay = "0"
duplicate = 1

motor1_pos = get_pos_list(start=92.4, n_step=12, step_size=0.1)
motor2_pos = get_pos_list(start=5, n_step=3, step_size=0.1)
motor3_pos = get_pos_list(start=20, n_step=2, step_size=0.1)

In [4]:
motor1_pos = sorted(motor1_pos*duplicate)

In [5]:
output_file_name = fname+".csv"
fname_PV = "CG1D:Det:A1:TIFF1:FileName"
camera = "CCD 600"
header_list = ["CG1D:Det:A1:TIFF1:FileName"]
if motor1 is not None:
    header_list.append(motor1)
if motor2 is not None:
    header_list.append(motor2)
if motor3 is not None:
    header_list.append(motor3)
header_list.append("CG1D:Det:A1:AcquireTime")
header_list.append("Delay")
header_list.append("Camera")

In [6]:
scan_dict = {}
_motor1_pos_list = []
_motor2_pos_list = []
_motor3_pos_list = []
_fname_list = []
if motor1 is not None:
    for pos1 in motor1_pos:
        if motor2 is None:
            _motor1_pos_list.append(pos1)
            _motor2_pos_list.append(None)
            _motor3_pos_list.append(None)
            _fname_list.append(fname+"_"+str(pos1))
        else:
            _motor1_pos_list.append(pos1)
            if motor3 is None:
                for pos2 in motor2_pos:
                    _motor2_pos_list.append(pos2)
                    _fname_list.append(fname+"_"+str(pos1)+"_"+str(pos2))
                _motor1_pos_list = _motor1_pos_list + (['']*(len(motor2_pos)-1))
            else:
                for pos2 in motor2_pos:
                    _motor2_pos_list.append(pos2)
                    for pos3 in motor3_pos:
                        _motor3_pos_list.append(pos3)
                        _fname_list.append(fname+"_"+str(pos1)+"_"+str(pos2)+"_"+str(pos3))
                    _motor2_pos_list = _motor2_pos_list + (['']*(len(motor3_pos)-1))
                _motor1_pos_list = _motor1_pos_list + (['']*(len(motor2_pos) + len(motor3_pos)))
scan_dict[fname_PV] = _fname_list
if motor1 is not None:
    scan_dict[motor1] = _motor1_pos_list[:]
if motor2 is not None:
    scan_dict[motor2] = _motor2_pos_list[:]
if motor3 is not None:
    scan_dict[motor3] = _motor3_pos_list[:]

# pprint.pprint(scan_dict)
# print(len(scan_dict[motor1]))
# print(len(scan_dict[motor2]))
# print(len(scan_dict[motor3]))
row_num = len(scan_dict[fname_PV])
scan_dict['Delay'] = [delay]*row_num
scan_dict['Camera'] = [camera]*row_num

scan_df = pd.DataFrame.from_dict(scan_dict)
scan_df
        

,CG1D:Det:A1:TIFF1:FileName,CG1D:Mot:G0:X,Delay,Camera
0,G0_stepping_92.4,92.4,0,CCD 600
1,G0_stepping_92.5,92.5,0,CCD 600
2,G0_stepping_92.6,92.6,0,CCD 600
3,G0_stepping_92.7,92.7,0,CCD 600
4,G0_stepping_92.8,92.8,0,CCD 600
5,G0_stepping_92.9,92.9,0,CCD 600
6,G0_stepping_93.0,93.0,0,CCD 600
7,G0_stepping_93.1,93.1,0,CCD 600
8,G0_stepping_93.2,93.2,0,CCD 600
9,G0_stepping_93.3,93.3,0,CCD 600


In [8]:
scan_df.to_csv(output_file_name, index=False)